# **House sale prices**

## Objectives

* Train a machine learning model to predict house prices
* Use the prepared and transformed data created in previous steps (DataCollection, DataCleaning, and FeatureEngineering)
* Optimize hyperparameters to improve the model’s performance

## Inputs

* 

## Outputs

*